## Load Data

In [98]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
import numpy as np
import random

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [99]:
batch_size = 512
num_epochs = 15

train_dataset = FashionMNIST('./data', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

In [100]:
import torch
import torch.nn as nn

class SwiGLU(nn.Module):
    def __init__(self):
        super(SwiGLU, self).__init__()
        self.sigmoid = nn.Sigmoid()  # Khởi tạo hàm Sigmoid

    def forward(self, x):
        a, b = x.chunk(2, dim=1)  # Chia tensor x thành 2 phần dọc theo chiều dim=1 (một nửa cho mỗi phần)
        return a * self.sigmoid(b)  # Kết hợp với hàm sigmoid cho phần b và nhân với a


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super(MLP, self).__init__()
        self.hidden_dims = hidden_dims
        self.layer1 = nn.Linear(input_dims, hidden_dims*2)
        self.layer2 = nn.Linear(hidden_dims, hidden_dims*2)
        self.layer3 = nn.Linear(hidden_dims, hidden_dims*2)
        self.output = nn.Linear(hidden_dims, output_dims)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0.0) 


    def forward(self, x):
        x = nn.Flatten()(x)
        x = self.layer1(x)
        x = nn.BatchNorm1d(self.hidden_dims*2, device=device)(x)
        x = SwiGLU()(x)
        skip = x

        x = self.layer2(x)
        x = nn.BatchNorm1d(self.hidden_dims*2, device=device)(x)
        x = SwiGLU()(x)
        x = self.layer3(x)
        x = nn.BatchNorm1d(self.hidden_dims*2, device=device)(x)
        x = SwiGLU()(x)

        x = skip + x
        out = self.output(x)

        return out

In [102]:
model = MLP(input_dims=784, hidden_dims=128, output_dims=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [103]:
for epoch in range(num_epochs):    
    t_loss = 0
    t_acc = 0
    cnt = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        t_loss += loss.item()
        t_acc += (torch.argmax(outputs, 1) == y).sum().item()
        cnt += len(y)

    t_loss /= len(train_loader)
    t_acc /= cnt
    print(f"Epoch {epoch+1}/{num_epochs}, Train_Loss: {t_loss:.4f}, Train_Acc: {t_acc:.4f}")

Epoch 1/15, Train_Loss: 0.5146, Train_Acc: 0.8195
Epoch 2/15, Train_Loss: 0.3418, Train_Acc: 0.8763
Epoch 3/15, Train_Loss: 0.2937, Train_Acc: 0.8938
Epoch 4/15, Train_Loss: 0.2665, Train_Acc: 0.9028
Epoch 5/15, Train_Loss: 0.2472, Train_Acc: 0.9095
Epoch 6/15, Train_Loss: 0.2230, Train_Acc: 0.9190
Epoch 7/15, Train_Loss: 0.2062, Train_Acc: 0.9250
Epoch 8/15, Train_Loss: 0.1938, Train_Acc: 0.9307
Epoch 9/15, Train_Loss: 0.1836, Train_Acc: 0.9338
Epoch 10/15, Train_Loss: 0.1659, Train_Acc: 0.9403
Epoch 11/15, Train_Loss: 0.1570, Train_Acc: 0.9425
Epoch 12/15, Train_Loss: 0.1443, Train_Acc: 0.9474
Epoch 13/15, Train_Loss: 0.1348, Train_Acc: 0.9520
Epoch 14/15, Train_Loss: 0.1247, Train_Acc: 0.9549
Epoch 15/15, Train_Loss: 0.1205, Train_Acc: 0.9570


In [104]:
import torch
import torch.nn as nn

class SwiGLU(nn.Module):
    def __init__(self):
        super(SwiGLU, self).__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        a, b = x.chunk(2, dim=1)  # Chia x thành 2 phần dọc theo chiều dim=1
        return a * self.sigmoid(b)  # Nhân phần a với sigmoid của phần b

# Định nghĩa lớp MLP
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dims, hidden_dims*2)
        self.fc2 = nn.Linear(hidden_dims, hidden_dims*2)
        self.output = nn.Linear(hidden_dims, output_dims)

    def forward(self, out):
        out = nn.Flatten()(out)  # Làm phẳng đầu vào
        out = self.fc1(out)
        out = SwiGLU()(out)  # Sử dụng SwiGLU
        out = self.fc2(out)
        out = SwiGLU()(out)  # Sử dụng SwiGLU
        out = self.output(out)
        return out

# Kiểm tra với một tensor giả
input_dims = 64  # Kích thước đầu vào
hidden_dims = 128  # Kích thước lớp ẩn
output_dims = 10  # Kích thước đầu ra (ví dụ cho phân loại 10 lớp)

# Khởi tạo mô hình
model = MLP(input_dims, hidden_dims, output_dims)

# Tạo một batch giả (ví dụ, 32 mẫu đầu vào, mỗi mẫu có 64 đặc trưng)
X = torch.randn(32, input_dims)

# Tiến hành kiểm tra mô hình với dữ liệu giả
outputs = model(X)

# In kết quả đầu ra
print(outputs.shape)  # Kết quả đầu ra phải có shape (32, 10) vì output_dims là 10


torch.Size([32, 10])
